In [9]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import com.microsoft.spark.fabric
from com.microsoft.spark.fabric.Constants import Constants  

StatementMeta(, bfcaee27-0587-40ae-a852-05d52e9990e6, 11, Finished, Available, Finished)

In [10]:
# Step 1: Read from Fabric Warehouse table
df = spark.read.synapsesql("wrhouse.dbo.accounts")
df.show()

StatementMeta(, bfcaee27-0587-40ae-a852-05d52e9990e6, 12, Finished, Available, Finished)

+-----------+-------+------------+----------+
|customer_id|balance|account_type|account_id|
+-----------+-------+------------+----------+
|         45| 1000.5|     Savings|       1.0|
|         12|2500.75|    Checking|       2.0|
|         78| 1500.0|     Savings|       3.0|
|         34|3000.25|    Checking|       4.0|
|         56|  500.0|     Savings|       5.0|
|         23| 1200.5|    Checking|       6.0|
|         89| 800.75|     Savings|       7.0|
|         67| 2200.0|    Checking|       8.0|
|         14| 900.25|     Savings|       9.0|
|         92| 1800.5|    Checking|      10.0|
|          3|1100.75|     Savings|      11.0|
|         81| 2700.0|    Checking|      12.0|
|         29|1300.25|     Savings|      13.0|
|         64| 3200.5|    Checking|      14.0|
|         47| 700.75|     Savings|      15.0|
|         18| 1400.0|    Checking|      16.0|
|         99| 600.25|     Savings|      17.0|
|          5| 1600.5|    Checking|      18.0|
|         76| 400.75|     Savings|

In [11]:
create_table_query = """
CREATE TABLE if not exists Account_SCD (
    account_id int,
    customer_id int,
    account_type STRING,
    balance FLOAT,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Files/Gold_layer/Account_SCD'
"""

# Execute
spark.sql(create_table_query)



StatementMeta(, bfcaee27-0587-40ae-a852-05d52e9990e6, 13, Finished, Available, Finished)

DataFrame[]

In [12]:
from pyspark.sql.functions import *
df_src1= df.withColumn("hash_key",crc32(concat(*df.columns)))
display(df_src1)

StatementMeta(, bfcaee27-0587-40ae-a852-05d52e9990e6, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 997a1071-ffad-4a56-9231-283752c51fb1)

In [13]:
from delta.tables import DeltaTable

target_path = "Files/Gold_layer/Account_SCD"  
delta_target = DeltaTable.forPath(spark, target_path)
delta_target.toDF().show()

StatementMeta(, bfcaee27-0587-40ae-a852-05d52e9990e6, 15, Finished, Available, Finished)

+----------+-----------+------------+-------+----------+----------+--------------------+----------+--------------------+
|account_id|customer_id|account_type|balance|  hash_key|created_by|        created_date|updated_by|        updated_date|
+----------+-----------+------------+-------+----------+----------+--------------------+----------+--------------------+
|         1|         45|     Savings| 1000.5|1081396551|databricks|2025-05-04 18:11:...|databricks|2025-05-04 18:11:...|
|         2|         12|    Checking|2500.75|1322625823|databricks|2025-05-04 18:11:...|databricks|2025-05-04 18:11:...|
|         3|         78|     Savings| 1500.0|4179129744|databricks|2025-05-04 18:11:...|databricks|2025-05-04 18:11:...|
|         4|         34|    Checking|3000.25| 463029068|databricks|2025-05-04 18:11:...|databricks|2025-05-04 18:11:...|
|         5|         56|     Savings|  500.0| 787317497|databricks|2025-05-04 18:11:...|databricks|2025-05-04 18:11:...|
|         6|         23|    Chec

In [6]:
df_src1=df_src1.alias("src").join(delta_target.toDF().alias("tgt"),((col("src.account_id")==col("tgt.account_id"))&(col("src.hash_key")==col("tgt.hash_key"))),"anti").select(col("src.*"))
df_src1.show()

StatementMeta(, bfcaee27-0587-40ae-a852-05d52e9990e6, 8, Finished, Available, Finished)

+-----------+-------+------------+----------+--------+
|customer_id|balance|account_type|account_id|hash_key|
+-----------+-------+------------+----------+--------+
+-----------+-------+------------+----------+--------+



In [8]:
from pyspark.sql.functions import col

delta_target.alias("tgt").merge(df_src1.alias("src"),"tgt.account_id = src.account_id")\
    .whenMatchedUpdate(set={"tgt.account_id":"src.account_id","tgt.customer_id":"src.customer_id","tgt.account_type":"src.account_type","tgt.balance":"src.balance","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.account_id":"src.account_id","tgt.customer_id":"src.customer_id","tgt.account_type":"src.account_type","tgt.balance":"src.balance","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
display(spark.read.format("delta").option("header","true").load(target_path))

StatementMeta(, bfcaee27-0587-40ae-a852-05d52e9990e6, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5fafcfcd-74a8-45af-a1b1-165ea2ebe2a3)

In [ ]:
#Customer Table
df_cust = spark.read.synapsesql("wrhouse.dbo.customers")

StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 11, Finished, Available, Finished)

In [ ]:
create_table_query = """
CREATE TABLE if not exists Customer_SCD1 (
    customer_id int,
    first_name string,
    last_name string,
    address string,
    city string,
    state string,
    zip string,
    hash_key BIGINT,
    created_by STRING,
    created_date TIMESTAMP,
    updated_by STRING,
    updated_date TIMESTAMP
)
USING DELTA
LOCATION 'Files/Gold_layer/Customer_SCD1'
"""
# Execute the query to create the table
spark.sql(create_table_query)

StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 12, Finished, Available, Finished)

DataFrame[]

In [ ]:
from delta.tables import DeltaTable

target_path_cust = "Files/Gold_layer/Customer_SCD1"  
delta_target_cust = DeltaTable.forPath(spark, target_path_cust)
delta_target_cust.toDF().show()

StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 13, Finished, Available, Finished)

+-----------+-----------+---------+---------------+-------------+-----+------+----------+----------+--------------------+----------+--------------------+
|customer_id| first_name|last_name|        address|         city|state|   zip|  hash_key|created_by|        created_date|updated_by|        updated_date|
+-----------+-----------+---------+---------------+-------------+-----+------+----------+----------+--------------------+----------+--------------------+
|          1|       John|      Doe|     123 Elm St|      Toronto|   ON|M4B1B3|1737850921|databricks|2025-05-04 19:16:...|databricks|2025-05-04 19:16:...|
|          2|       Jane|    Smith|  456 Maple Ave|       Ottawa|   ON|K1A0B1|3806675518|databricks|2025-05-04 19:16:...|databricks|2025-05-04 19:16:...|
|          3|    Michael|  Johnson|     789 Oak Dr|     Montreal|   QC|H1A1A1|1389944316|databricks|2025-05-04 19:16:...|databricks|2025-05-04 19:16:...|
|          4|      Emily|    Davis|    101 Pine Rd|      Calgary|   AB|T2A0A

In [ ]:
from pyspark.sql.functions import *
df_src1_cust= df_cust.withColumn("hash_key",crc32(concat(*df_cust.columns)))
display(df_src1_cust)

StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 14e16608-c510-4985-aff8-987c7ec75de7)

In [ ]:
df_src1_cust1=df_src1_cust.alias("src").join(delta_target_cust.toDF().alias("tgt"), ((col("src.customer_id") == col("tgt.customer_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_src1_cust1.show()

StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 15, Finished, Available, Finished)

+----------+----------+-----------+----+-----+---------+--------------+--------+
|first_name|      city|customer_id| zip|state|last_name|       address|hash_key|
+----------+----------+-----------+----+-----+---------+--------------+--------+
|   William|Haileybury|       87.0|NULL| NULL| McDonald|8686 Maple Ave|    NULL|
+----------+----------+-----------+----+-----+---------+--------------+--------+



In [ ]:
delta_target_cust.alias("tgt").merge(df_src1_cust1.alias("src"),"tgt.customer_id = src.customer_id")\
    .whenMatchedUpdate(set={"tgt.customer_id":"src.customer_id","tgt.first_name":"src.first_name","tgt.last_name":"src.last_name","tgt.address":"src.address","tgt.city":"src.city","tgt.state":"src.state","tgt.zip":"src.zip","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.customer_id":"src.customer_id","tgt.first_name":"src.first_name","tgt.last_name":"src.last_name","tgt.address":"src.address","tgt.city":"src.city","tgt.state":"src.state","tgt.zip":"src.zip","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
     
display(spark.read.format("delta").option("header","true").load(target_path_cust))



StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2390f8df-bff0-4c50-bed1-8289963b1a27)

In [ ]:
#loan_paymnets
df = spark.read.synapsesql("wrhouse.dbo.loan_payments")
df.show()

StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 17, Finished, Available, Finished)

+----------+------------+--------------+-------+
|payment_id|payment_date|payment_amount|loan_id|
+----------+------------+--------------+-------+
|       1.0|  2024-01-01|         100.0|     45|
|       2.0|  2024-01-02|         150.0|     23|
|       3.0|  2024-01-03|         200.0|     67|
|       4.0|  2024-01-04|         250.0|     89|
|       5.0|  2024-01-05|         300.0|     12|
|       6.0|  2024-01-06|         350.0|     34|
|       7.0|  2024-01-07|         400.0|     56|
|       8.0|  2024-01-08|         450.0|     78|
|       9.0|  2024-01-09|         500.0|     90|
|      10.0|  2024-01-10|         550.0|     11|
|      11.0|  2024-01-11|         600.0|     22|
|      12.0|  2024-01-12|         650.0|     33|
|      13.0|  2024-01-13|         700.0|     44|
|      14.0|  2024-01-14|         750.0|     55|
|      15.0|  2024-01-15|         800.0|     66|
|      16.0|  2024-01-16|         850.0|     77|
|      17.0|  2024-01-17|         900.0|     88|
|      18.0|  2024-0

In [ ]:
create_table_query = """
CREATE TABLE if not exists Loan_Payment (
  payment_id int,
  loan_id int,
  payment_date timestamp,
  payment_amount float,
  hash_key bigint,
  created_by string,
  created_date timestamp,
  updated_by string,
  updated_date timestamp
)
USING DELTA
LOCATION 'Files/Gold_layer/Loan_Payment_SCD'
"""
# Execute the query to create the table
spark.sql(create_table_query)
     

StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 18, Finished, Available, Finished)

DataFrame[]

In [ ]:
from delta.tables import DeltaTable

target_path_lp = "Files/Gold_layer/Loan_Payment_SCD"  
delta_target_lp = DeltaTable.forPath(spark, target_path_lp)
delta_target_lp.toDF().show()

StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 19, Finished, Available, Finished)

+----------+-------+-------------------+--------------+----------+----------+--------------------+----------+--------------------+
|payment_id|loan_id|       payment_date|payment_amount|  hash_key|created_by|        created_date|updated_by|        updated_date|
+----------+-------+-------------------+--------------+----------+----------+--------------------+----------+--------------------+
|         1|     45|2024-01-01 00:00:00|         100.0|1831072645|databricks|2025-05-04 19:47:...|databricks|2025-05-04 19:47:...|
|         2|     23|2024-01-02 00:00:00|         150.0|2750491308|databricks|2025-05-04 19:47:...|databricks|2025-05-04 19:47:...|
|         3|     67|2024-01-03 00:00:00|         200.0| 135830343|databricks|2025-05-04 19:47:...|databricks|2025-05-04 19:47:...|
|         4|     89|2024-01-04 00:00:00|         250.0|3396314615|databricks|2025-05-04 19:47:...|databricks|2025-05-04 19:47:...|
|         5|     12|2024-01-05 00:00:00|         300.0|3540543209|databricks|2025-0

In [ ]:
from pyspark.sql.functions import *
df_src1_lp= df.withColumn("hash_key",crc32(concat(*df.columns)))
display(df_src1_lp)

StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b573e1af-57be-4f61-81e1-40b6829489f4)

In [ ]:

df_src1_lp1=df_src1_lp.alias("src").join(delta_target_lp.toDF().alias("tgt"), ((col("src.payment_id") == col("tgt.payment_id")) & (col("src.hash_key") == col("tgt.hash_key"))), "anti").select(col("src.*"))
df_src1_lp1.show()

StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 21, Finished, Available, Finished)

+----------+------------+--------------+-------+--------+
|payment_id|payment_date|payment_amount|loan_id|hash_key|
+----------+------------+--------------+-------+--------+
+----------+------------+--------------+-------+--------+



In [ ]:
delta_target_lp.alias("tgt").merge(df_src1_lp1.alias("src"),"tgt.payment_id = src.payment_id")\
    .whenMatchedUpdate(set={"tgt.payment_id":"src.payment_id","tgt.loan_id":"src.loan_id","tgt.payment_date":"src.payment_date","tgt.payment_amount":"src.payment_amount","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks_Updated")})\
        .whenNotMatchedInsert(values={"tgt.payment_id":"src.payment_id","tgt.loan_id":"src.loan_id","tgt.payment_date":"src.payment_date","tgt.payment_amount":"src.payment_amount","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.created_by":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updated_by":lit("databricks")}).execute()
display(spark.read.format("delta").option("header","true").load(target_path_lp))

StatementMeta(, 57a4b3b3-8002-4120-96b9-024fc497aa0f, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c4d6e1a3-3a7b-44ae-b784-d008c8988093)